In [3]:
import pandas as pd

In [19]:
import numpy as np

In [4]:
def read_test_data():
    Dfs = []
    train_df = []
    coords = pd.read_csv("data_phase_one/coords.csv", header=None)
    for name in ["mslp", "sst", "ssta", "t2m"]:
        df = pd.read_csv(f"./data_phase_one/test_input_{name}_phase_one.csv", header=None)
        print("Shape df", df.shape)
        Dfs.append(df)
        #Dfs.append(df.T.reset_index(drop=True))
    j = 0
    for i in range(0, df.shape[0], 12):
        feat = pd.concat([coords, Dfs[0].iloc[i:i+12], Dfs[1].iloc[i:i+12], Dfs[2].iloc[i:i+12], Dfs[3].iloc[i:i+12]], axis=0).reset_index(drop=True)
        feat = feat.T.reset_index(drop=True)
        feat.columns = [f"feat_{k}" for k in range(feat.shape[1])]
        if j < 3:
            print(feat.columns)
        train_df.append(feat)
        j += 1
    train_df = pd.concat(train_df, axis=0)
    return train_df

In [5]:
test_data = read_test_data()

Shape df (852, 5774)
Shape df (852, 5774)
Shape df (852, 5774)
Shape df (852, 5774)
Index(['feat_0', 'feat_1', 'feat_2', 'feat_3', 'feat_4', 'feat_5', 'feat_6',
       'feat_7', 'feat_8', 'feat_9', 'feat_10', 'feat_11', 'feat_12',
       'feat_13', 'feat_14', 'feat_15', 'feat_16', 'feat_17', 'feat_18',
       'feat_19', 'feat_20', 'feat_21', 'feat_22', 'feat_23', 'feat_24',
       'feat_25', 'feat_26', 'feat_27', 'feat_28', 'feat_29', 'feat_30',
       'feat_31', 'feat_32', 'feat_33', 'feat_34', 'feat_35', 'feat_36',
       'feat_37', 'feat_38', 'feat_39', 'feat_40', 'feat_41', 'feat_42',
       'feat_43', 'feat_44', 'feat_45', 'feat_46', 'feat_47', 'feat_48',
       'feat_49'],
      dtype='object')
Index(['feat_0', 'feat_1', 'feat_2', 'feat_3', 'feat_4', 'feat_5', 'feat_6',
       'feat_7', 'feat_8', 'feat_9', 'feat_10', 'feat_11', 'feat_12',
       'feat_13', 'feat_14', 'feat_15', 'feat_16', 'feat_17', 'feat_18',
       'feat_19', 'feat_20', 'feat_21', 'feat_22', 'feat_23', 'feat_24

In [9]:
test_data.shape

(409954, 50)

In [7]:
import joblib

In [8]:
lgb_model = joblib.load("./trained_models/lgb_model.joblib")
catb_model = joblib.load("./trained_models/catb_model.joblib")

In [47]:
y_pred1 = lgb_model.predict(test_data.values)
y_pred2 = catb_model.predict(test_data.values)

In [48]:
#y_pred = 0.8*y_pred1 + 0.2*y_pred2 ==> -0.01
#y_pred = 0.2*y_pred1 + 0.8*y_pred2 ==> -0.06
#y_pred = 0.55*y_pred1 + 0.45*y_pred2 ==> -0.03
#y_pred = 0.95*y_pred1 + 0.05*y_pred2 ==> -0.0012

In [49]:
y_pred = 0.99*y_pred1 - 0.01*y_pred2

In [50]:
pred_df = pd.DataFrame(y_pred.reshape(71, 5774))

In [25]:
pred_df

,0,1,2,3,4,5,6,7,8,9,...,5764,5765,5766,5767,5768,5769,5770,5771,5772,5773
0,0.543218,0.892128,0.673486,0.106131,-0.085835,-0.085108,-0.095640,-0.113741,-0.085838,0.083399,...,0.605443,0.162431,-0.357387,-0.429471,0.022132,0.052346,-0.230947,-0.113145,0.411573,-0.076587
1,-0.752504,-0.750676,-0.641753,-0.866679,-1.029404,-0.981070,-0.939580,-0.806037,-0.804079,-0.751553,...,-0.571098,-0.642855,-0.261020,-0.251433,-0.250770,-0.239764,-0.334639,-0.141069,-0.170265,1.274184
2,-0.346026,-0.481755,-0.442041,-0.404317,-0.367506,-0.260691,-0.199471,-0.180590,-0.179121,-0.160209,...,-0.282619,-0.325036,-0.067722,0.151273,0.411442,0.545201,0.718478,0.547036,0.676546,1.888504
3,-0.379562,-0.391677,-0.280689,-0.169402,-0.105036,-0.067724,-0.058398,-0.057791,-0.048719,-0.040428,...,0.073129,0.251514,0.309954,0.410071,0.472511,0.474185,0.293589,0.357880,0.520274,0.680204
4,-0.345587,-0.496134,-0.392927,-0.682078,-0.729394,-0.556527,-0.468781,-0.469316,-0.571194,-0.590483,...,-0.573663,-0.510821,-0.302735,-0.239908,-0.179688,-0.123260,-0.209933,-0.031222,0.031816,-0.024249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,-0.367889,-0.229901,-0.210249,-0.336071,-0.272531,-0.314066,-0.324099,-0.190065,-0.170328,-0.240625,...,-0.429815,-0.540388,0.002546,-0.085812,-0.047754,-0.057541,0.095670,-0.029919,0.357892,0.357665
67,-0.057155,-0.040248,-0.077106,-0.141001,-0.123018,-0.067465,-0.047677,-0.024522,-0.019946,-0.015060,...,0.898701,0.674410,0.410775,0.265520,0.211475,0.211509,0.199128,-0.122174,0.252339,-0.454131
68,0.720059,0.636381,0.603668,0.356040,0.322428,0.093609,0.262521,0.290947,0.104523,-0.076375,...,0.212093,0.175014,0.187115,0.358011,0.237323,0.251538,0.410188,0.279165,0.237242,-0.169428
69,-0.271111,-0.249971,-0.189865,-0.114242,-0.076236,-0.056890,-0.048403,-0.031374,-0.030249,-0.024693,...,0.393371,0.324809,0.295888,0.359299,0.391767,0.002602,0.279338,0.225237,0.211407,0.768786


In [51]:
pred_df

,0,1,2,3,4,5,6,7,8,9,...,5764,5765,5766,5767,5768,5769,5770,5771,5772,5773
0,0.319319,0.572304,0.418131,0.034194,-0.094866,-0.091535,-0.104013,-0.113955,-0.094865,0.015737,...,0.368848,0.068073,-0.277825,-0.327076,-0.022456,-0.001951,-0.195230,-0.110399,0.236758,-0.088666
1,-0.540650,-0.532562,-0.468001,-0.617580,-0.730811,-0.695809,-0.667230,-0.576006,-0.566357,-0.532551,...,-0.413339,-0.467987,-0.212497,-0.209054,-0.205658,-0.195119,-0.258905,-0.129857,-0.152265,0.786975
2,-0.270149,-0.358684,-0.335569,-0.309768,-0.281496,-0.212501,-0.172070,-0.162396,-0.155615,-0.145782,...,-0.230072,-0.255150,-0.085344,0.060464,0.236760,0.319294,0.446395,0.328054,0.418093,1.387373
3,-0.289568,-0.297284,-0.222680,-0.148917,-0.110500,-0.085344,-0.078941,-0.075908,-0.069775,-0.066587,...,0.008415,0.130433,0.168838,0.230307,0.271216,0.279313,0.154065,0.200530,0.310194,0.418047
4,-0.266495,-0.368202,-0.301589,-0.488352,-0.524639,-0.408516,-0.349370,-0.354046,-0.413338,-0.436558,...,-0.424700,-0.378120,-0.236801,-0.195118,-0.155608,-0.120452,-0.178765,-0.060669,-0.015865,-0.055139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,-0.281491,-0.191844,-0.182278,-0.262826,-0.222782,-0.243843,-0.251431,-0.165498,-0.152264,-0.198663,...,-0.327072,-0.398014,-0.035684,-0.094866,-0.066495,-0.075912,0.026799,-0.055068,0.200530,0.200533
67,-0.075916,-0.066589,-0.091635,-0.129858,-0.120455,-0.085347,-0.066496,-0.055136,-0.049660,-0.049721,...,0.572141,0.404630,0.230298,0.135306,0.100222,0.100221,0.091833,-0.116939,0.130423,-0.340071
68,0.446375,0.379934,0.368871,0.194895,0.174015,0.023123,0.135343,0.154098,0.030241,-0.088668,...,0.104416,0.079860,0.087763,0.200528,0.117303,0.130433,0.230306,0.144841,0.117304,-0.148916
69,-0.219242,-0.202195,-0.165500,-0.117038,-0.088670,-0.072530,-0.069779,-0.060667,-0.055064,-0.055134,...,0.224635,0.173985,0.164176,0.206223,0.218770,-0.035685,0.144838,0.113099,0.100222,0.462331


In [52]:
pred_df.to_csv("./predictions.csv", index=False, header=False)

In [3]:
import pandas as pd

In [19]:
import numpy as np

In [4]:
def read_test_data():
    Dfs = []
    train_df = []
    coords = pd.read_csv("data_phase_one/coords.csv", header=None)
    for name in ["mslp", "sst", "ssta", "t2m"]:
        df = pd.read_csv(f"./data_phase_one/test_input_{name}_phase_one.csv", header=None)
        print("Shape df", df.shape)
        Dfs.append(df)
        #Dfs.append(df.T.reset_index(drop=True))
    j = 0
    for i in range(0, df.shape[0], 12):
        feat = pd.concat([coords, Dfs[0].iloc[i:i+12], Dfs[1].iloc[i:i+12], Dfs[2].iloc[i:i+12], Dfs[3].iloc[i:i+12]], axis=0).reset_index(drop=True)
        feat = feat.T.reset_index(drop=True)
        feat.columns = [f"feat_{k}" for k in range(feat.shape[1])]
        if j < 3:
            print(feat.columns)
        train_df.append(feat)
        j += 1
    train_df = pd.concat(train_df, axis=0)
    return train_df

In [5]:
test_data = read_test_data()

Shape df (852, 5774)
Shape df (852, 5774)
Shape df (852, 5774)
Shape df (852, 5774)
Index(['feat_0', 'feat_1', 'feat_2', 'feat_3', 'feat_4', 'feat_5', 'feat_6',
       'feat_7', 'feat_8', 'feat_9', 'feat_10', 'feat_11', 'feat_12',
       'feat_13', 'feat_14', 'feat_15', 'feat_16', 'feat_17', 'feat_18',
       'feat_19', 'feat_20', 'feat_21', 'feat_22', 'feat_23', 'feat_24',
       'feat_25', 'feat_26', 'feat_27', 'feat_28', 'feat_29', 'feat_30',
       'feat_31', 'feat_32', 'feat_33', 'feat_34', 'feat_35', 'feat_36',
       'feat_37', 'feat_38', 'feat_39', 'feat_40', 'feat_41', 'feat_42',
       'feat_43', 'feat_44', 'feat_45', 'feat_46', 'feat_47', 'feat_48',
       'feat_49'],
      dtype='object')
Index(['feat_0', 'feat_1', 'feat_2', 'feat_3', 'feat_4', 'feat_5', 'feat_6',
       'feat_7', 'feat_8', 'feat_9', 'feat_10', 'feat_11', 'feat_12',
       'feat_13', 'feat_14', 'feat_15', 'feat_16', 'feat_17', 'feat_18',
       'feat_19', 'feat_20', 'feat_21', 'feat_22', 'feat_23', 'feat_24

In [9]:
test_data.shape

(409954, 50)

In [7]:
import joblib

In [8]:
lgb_model = joblib.load("./trained_models/lgb_model.joblib")
catb_model = joblib.load("./trained_models/catb_model.joblib")

In [47]:
y_pred1 = lgb_model.predict(test_data.values)
y_pred2 = catb_model.predict(test_data.values)

In [48]:
#y_pred = 0.2*y_pred1 + 0.8*y_pred2 ==> -0.06
#y_pred = 0.55*y_pred1 + 0.45*y_pred2 ==> -0.03
#y_pred = 0.8*y_pred1 + 0.2*y_pred2 ==> -0.01
#y_pred = 0.95*y_pred1 + 0.05*y_pred2 ==> -0.0012
#y_pred = 0.99*y_pred1 - 0.01*y_pred2 ==> 0.0032

In [49]:
y_pred = 0.99*y_pred1 - 0.01*y_pred2

In [50]:
pred_df = pd.DataFrame(y_pred.reshape(71, 5774))

In [25]:
pred_df

,0,1,2,3,4,5,6,7,8,9,...,5764,5765,5766,5767,5768,5769,5770,5771,5772,5773
0,0.543218,0.892128,0.673486,0.106131,-0.085835,-0.085108,-0.095640,-0.113741,-0.085838,0.083399,...,0.605443,0.162431,-0.357387,-0.429471,0.022132,0.052346,-0.230947,-0.113145,0.411573,-0.076587
1,-0.752504,-0.750676,-0.641753,-0.866679,-1.029404,-0.981070,-0.939580,-0.806037,-0.804079,-0.751553,...,-0.571098,-0.642855,-0.261020,-0.251433,-0.250770,-0.239764,-0.334639,-0.141069,-0.170265,1.274184
2,-0.346026,-0.481755,-0.442041,-0.404317,-0.367506,-0.260691,-0.199471,-0.180590,-0.179121,-0.160209,...,-0.282619,-0.325036,-0.067722,0.151273,0.411442,0.545201,0.718478,0.547036,0.676546,1.888504
3,-0.379562,-0.391677,-0.280689,-0.169402,-0.105036,-0.067724,-0.058398,-0.057791,-0.048719,-0.040428,...,0.073129,0.251514,0.309954,0.410071,0.472511,0.474185,0.293589,0.357880,0.520274,0.680204
4,-0.345587,-0.496134,-0.392927,-0.682078,-0.729394,-0.556527,-0.468781,-0.469316,-0.571194,-0.590483,...,-0.573663,-0.510821,-0.302735,-0.239908,-0.179688,-0.123260,-0.209933,-0.031222,0.031816,-0.024249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,-0.367889,-0.229901,-0.210249,-0.336071,-0.272531,-0.314066,-0.324099,-0.190065,-0.170328,-0.240625,...,-0.429815,-0.540388,0.002546,-0.085812,-0.047754,-0.057541,0.095670,-0.029919,0.357892,0.357665
67,-0.057155,-0.040248,-0.077106,-0.141001,-0.123018,-0.067465,-0.047677,-0.024522,-0.019946,-0.015060,...,0.898701,0.674410,0.410775,0.265520,0.211475,0.211509,0.199128,-0.122174,0.252339,-0.454131
68,0.720059,0.636381,0.603668,0.356040,0.322428,0.093609,0.262521,0.290947,0.104523,-0.076375,...,0.212093,0.175014,0.187115,0.358011,0.237323,0.251538,0.410188,0.279165,0.237242,-0.169428
69,-0.271111,-0.249971,-0.189865,-0.114242,-0.076236,-0.056890,-0.048403,-0.031374,-0.030249,-0.024693,...,0.393371,0.324809,0.295888,0.359299,0.391767,0.002602,0.279338,0.225237,0.211407,0.768786


In [51]:
pred_df

,0,1,2,3,4,5,6,7,8,9,...,5764,5765,5766,5767,5768,5769,5770,5771,5772,5773
0,0.319319,0.572304,0.418131,0.034194,-0.094866,-0.091535,-0.104013,-0.113955,-0.094865,0.015737,...,0.368848,0.068073,-0.277825,-0.327076,-0.022456,-0.001951,-0.195230,-0.110399,0.236758,-0.088666
1,-0.540650,-0.532562,-0.468001,-0.617580,-0.730811,-0.695809,-0.667230,-0.576006,-0.566357,-0.532551,...,-0.413339,-0.467987,-0.212497,-0.209054,-0.205658,-0.195119,-0.258905,-0.129857,-0.152265,0.786975
2,-0.270149,-0.358684,-0.335569,-0.309768,-0.281496,-0.212501,-0.172070,-0.162396,-0.155615,-0.145782,...,-0.230072,-0.255150,-0.085344,0.060464,0.236760,0.319294,0.446395,0.328054,0.418093,1.387373
3,-0.289568,-0.297284,-0.222680,-0.148917,-0.110500,-0.085344,-0.078941,-0.075908,-0.069775,-0.066587,...,0.008415,0.130433,0.168838,0.230307,0.271216,0.279313,0.154065,0.200530,0.310194,0.418047
4,-0.266495,-0.368202,-0.301589,-0.488352,-0.524639,-0.408516,-0.349370,-0.354046,-0.413338,-0.436558,...,-0.424700,-0.378120,-0.236801,-0.195118,-0.155608,-0.120452,-0.178765,-0.060669,-0.015865,-0.055139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,-0.281491,-0.191844,-0.182278,-0.262826,-0.222782,-0.243843,-0.251431,-0.165498,-0.152264,-0.198663,...,-0.327072,-0.398014,-0.035684,-0.094866,-0.066495,-0.075912,0.026799,-0.055068,0.200530,0.200533
67,-0.075916,-0.066589,-0.091635,-0.129858,-0.120455,-0.085347,-0.066496,-0.055136,-0.049660,-0.049721,...,0.572141,0.404630,0.230298,0.135306,0.100222,0.100221,0.091833,-0.116939,0.130423,-0.340071
68,0.446375,0.379934,0.368871,0.194895,0.174015,0.023123,0.135343,0.154098,0.030241,-0.088668,...,0.104416,0.079860,0.087763,0.200528,0.117303,0.130433,0.230306,0.144841,0.117304,-0.148916
69,-0.219242,-0.202195,-0.165500,-0.117038,-0.088670,-0.072530,-0.069779,-0.060667,-0.055064,-0.055134,...,0.224635,0.173985,0.164176,0.206223,0.218770,-0.035685,0.144838,0.113099,0.100222,0.462331


In [52]:
pred_df.to_csv("./predictions.csv", index=False, header=False)